In [32]:
import pandas as pd
import numpy as np

### Insert all of your league settings and the path to your csv files with the data here.

In [33]:
pitching_sheet = '2018_Projected_P_Stats.csv'
C_sheet = '2018_Projected_C_Stats.csv'
firstbase_sheet = '2018_Projected_1B_Stats.csv'
secondbase_sheet = '2018_Projected_2B_Stats.csv'
SS_sheet = '2018_Projected_SS_Stats.csv'
thirdbase_sheet = '2018_Projected_3B_Stats.csv'
OF_sheet = '2018_Projected_OF_Stats.csv'
DH_sheet = '2018_Projected_DH_Stats.csv'


#INSERT SCORING FOR PITCHERS HERE
IP = 3
P_H = -1
K = 1
P_BB = -1 #include HBP
P_HR = 0
GS = 0
P_G = 0
ER = -2
W = 3
L = -3
SV = 5
P_SO = 0 #shutout
B = 0 #balk, negligable
PKO = 0 #pickoff, negligable
HD = 0 #hold
CG = 0 #complete game
QS = 0 #quality start
#################################

#INSERT SCORING FOR BATTERS HERE
B_G = 0
AB = 0
PA = 0
B_H = 0
single = 1
double = 2
triple = 3
B_HR = 4
R = 1
RBI = 1
B_BB = 1
SO = -1
HBP = 0
SB = 1
CS = -1
PO = 0
OFA = 0 #Outfield Assists
A = 0
E = 0
################################

#INSERT LEAGUE SETTINGS HERE
num_teams = 8
roster_size = 23 #exclude DL
dollars_per_team = 260
############################

In [34]:
pitchingDF = pd.read_csv(pitching_sheet)
C_DF = pd.read_csv(C_sheet)
firstbase_DF = pd.read_csv(firstbase_sheet)
secondbase_DF = pd.read_csv(secondbase_sheet)
thirdbase_DF = pd.read_csv(thirdbase_sheet)
SS_DF = pd.read_csv(SS_sheet)
OF_DF = pd.read_csv(OF_sheet)
DH_DF = pd.read_csv(DH_sheet)

In [35]:
#This function uses some historical data and various sites to predict the stats for scoring that are not given in the 
#predictions from FanGraphs: CG, SO, HLD, QS

def pitchingPredictions(pitchingDF):
    
    pitchingDF['QS'] = ((pitchingDF['GS']/(pitchingDF['ER']*(pitchingDF['GS']/pitchingDF['G'])))*(pitchingDF['IP']*(pitchingDF['GS']/pitchingDF['G']))*(((pitchingDF['GS']+pitchingDF['G'])/(2*pitchingDF['G']*2))**2))
    #CG and SO are so rare, we're not going to concern ourselves with predicting them.
    pitchingDF['HLD'] = ((pitchingDF['G']/pitchingDF['ERA'])+5).where(((pitchingDF['SV'] < 6) & (pitchingDF['GS'] < 4)), 0)
    pitchingDF = pitchingDF.fillna(0)
    return pitchingDF

pitchingDF = pitchingPredictions(pitchingDF)

In [36]:
def pitchingScoring(pitchingDF):
    
    pitchingDF['Points'] = (pitchingDF['IP']*IP) + (pitchingDF['H']*P_H) + (pitchingDF['SO']*K) + (pitchingDF['BB']*P_BB) + (pitchingDF['HR']*P_HR) + (pitchingDF['GS']*GS) + (pitchingDF['G']*P_G) + (pitchingDF['ER']*ER) + (pitchingDF['W']*W) + (pitchingDF['L']*L) + (pitchingDF['SV']*SV) + (pitchingDF['HLD']*HD) + (pitchingDF['QS']*QS)
    pitchingDF['Pos'] = 'P'
    pitchingDF = pitchingDF.sort_values(by=['Points'], ascending=False)
    return pitchingDF


In [37]:
def battingScoring(battingDF, Pos):
    
    battingDF['Points'] = (battingDF['G']*B_G) + (battingDF['AB']*AB) + (battingDF['PA']*PA) + (battingDF['H']*B_H) + ((battingDF['H']-(battingDF['2B']+battingDF['3B']+battingDF['HR']))*single) + (battingDF['2B']*double) + (battingDF['3B']*triple) + (battingDF['HR']*B_HR) + (battingDF['R']*R) + (battingDF['RBI']*RBI) + (battingDF['BB']*B_BB) + (battingDF['SO']*SO) + (battingDF['HBP']*HBP) + (battingDF['SB']*SB) + (battingDF['CS']*CS) + ((battingDF['Fld']*PO)*3)
    
    #All values gotten from running SQL scripts by position since 2005 by game average
    if(Pos == 'C'):
        battingDF['Points'] += (((battingDF['G']*6.6)*PO)+((battingDF['G']*0.46)*A)+((battingDF['G']*0.05)*E))*((battingDF['Fld']+5)/10)
    if(Pos == '1B'):
        battingDF['Points'] += (((battingDF['G']*7.4)*PO)+((battingDF['G']*0.56)*A)+((battingDF['G']*0.05)*E))*((battingDF['Fld']+5)/10)*(battingDF['Fld']/10)
    if(Pos == '2B'):
        battingDF['Points'] += (((battingDF['G']*1.7)*PO)+((battingDF['G']*2.5)*A)+((battingDF['G']*0.07)*E))*((battingDF['Fld']+5)/10)
    if(Pos == '3B'):
        battingDF['Points'] += (((battingDF['G']*0.6)*PO)+((battingDF['G']*1.6)*A)+((battingDF['G']*0.1)*E))*((battingDF['Fld']+5)/10)
    if(Pos == 'SS'):
        battingDF['Points'] += (((battingDF['G']*1.3)*PO)+((battingDF['G']*2.6)*A)+((battingDF['G']*0.1)*E))*((battingDF['Fld']+5)/10)
    if(Pos == 'OF'):
        battingDF['Points'] += (((battingDF['G']*1.8)*PO)+((battingDF['G']*0.05)*OFA)+((battingDF['G']*0.03)*E))*((battingDF['Fld']+5)/10)
    
    battingDF['Pos'] = Pos
    battingDF = battingDF.sort_values(by=['Points'], ascending=False)
    return battingDF

In [38]:
tempDF = pd.concat([battingScoring(C_DF, 'C'), battingScoring(firstbase_DF, '1B'), battingScoring(secondbase_DF, '2B'), battingScoring(thirdbase_DF, '3B'), battingScoring(SS_DF, 'SS'), battingScoring(OF_DF, 'OF'), battingScoring(DH_DF, 'DH'), pitchingScoring(pitchingPredictions(pitchingDF))], axis=0)

In [39]:
scoringDF = tempDF[['Name', 'Team', 'Pos', 'Points']]
scoringDF = scoringDF.sort_values(by=['Points'], ascending = False)
scoringDF = scoringDF.reset_index(drop=True)
scoringDF.drop(scoringDF.tail(7200).index,inplace=True)

In [40]:
def calculateValue(scoringDF):
    
    replacement_points = scoringDF['Points'][(num_teams*roster_size)+1]
    league_dollars = num_teams*dollars_per_team
    total_players = num_teams*roster_size
    
    total_points_above_replacement = 0
    for i in range(0,total_players):
        total_points_above_replacement += scoringDF['Points'][i]
    
    scoringDF['tempValue'] = (league_dollars * (scoringDF['Points']-replacement_points))/(total_points_above_replacement-(total_players*replacement_points))
    
    total_value_above_replacement = 0
    for i in range(0,total_players):
        total_value_above_replacement += scoringDF['tempValue'][i]
        
    scoringDF['Value'] = scoringDF['tempValue']*(league_dollars/total_value_above_replacement)
    
    scoringDF = scoringDF.drop(['tempValue'], axis=1)
    
    for i in range(0,total_players):
        total_points_above_replacement += scoringDF['Points'][i]
    
    return scoringDF
    
scoringDF = calculateValue(scoringDF.groupby(['Name']).mean())
scoringDF = scoringDF.sort_values(by=['Points'], ascending = False)


#calculateValue(scoringDF.groupby(['Name']).mean())

In [41]:
scoringDF.head(1000)

Points      Value
Name                                
Corey Kluber        521.0  35.554766
Mike Trout          521.0  35.554766
Max Scherzer        514.0  34.908316
Chris Sale          509.0  34.446566
Clayton Kershaw     503.0  33.892465
Mookie Betts        496.0  33.246015
Nolan Arenado       494.0  33.061315
Giancarlo Stanton   476.0  31.399014
Anthony Rizzo       475.0  31.306664
Manny Machado       471.0  30.937264
Joey Votto          465.0  30.383164
Bryce Harper        465.0  30.383164
Francisco Lindor    455.0  29.459663
Noah Syndergaard    454.0  29.367313
Jacob deGrom        448.0  28.813213
Jose Altuve         447.0  28.720863
Kris Bryant         436.0  27.705013
Justin Verlander    434.0  27.520313
Stephen Strasburg   433.0  27.427963
Paul Goldschmidt    432.0  27.335612
Chris Archer        432.0  27.335612
Josh Donaldson      427.0  26.873862
Luis Severino       423.0  26.504462
Charlie Blackmon    421.0  26.319762
Jose Quintana       414.0  25.673312
Freddie Freeman     411.0  25.396262
Carlos Correa       410.0  25.303912
Madison Bumgarner   408.0  25.119211
Edwin Encarnacion   408.0  25.119211
Rhys Hoskins        399.0  24.288061
...                   ...        ...
Jake McGee          143.0   0.646450
Walker Buehler      142.0   0.554100
Chad Bettis         142.0   0.554100
Yunel Escobar       142.0   0.554100
Matt Wieters        141.0   0.461750
Kyle Freeland       141.0   0.461750
Jose Leclerc        140.0   0.369400
Jacoby Ellsbury     140.0   0.369400
Steven Wright       140.0   0.369400
Carson Smith        139.0   0.277050
Nick Ahmed          138.0   0.184700
Yasmani Grandal     137.0   0.092350
Hector Rondon       136.0   0.000000
Adrian Gonzalez     136.0   0.000000
Tommy Kahnle        135.0  -0.092350
Darren O'Day        135.0  -0.092350
Tyson Ross          134.0  -0.184700
Matt Bush           134.0  -0.184700
Jason Castro        134.0  -0.184700
Pedro Strop         132.0  -0.369400
Tyler Lyons         132.0  -0.369400
Jose Reyes          132.0  -0.369400
A.J. Minter         132.0  -0.369400
Tyler Glasnow       131.0  -0.461750
Derek Fisher        130.0  -0.554100
Greg Holland        130.0  -0.554100
Josh Hader          130.0  -0.554100
Scott Alexander     129.0  -0.646450
Michael Feliz       129.0  -0.646450
Adam Engel          129.0  -0.646450

[490 rows x 2 columns]

In [42]:
scoringDF.to_csv('value_chart.csv')

In [52]:
OF_DF.iloc[0]

Name        Mike Trout
Team            Angels
G                  144
PA                 634
AB                 510
H                  157
2B                  32
3B                   3
HR                  38
R                  111
RBI                106
BB                 108
SO                 124
HBP                  9
SB                  20
CS                   9
-1                 NaN
AVG              0.308
OBP              0.434
SLG              0.608
OPS              1.042
wOBA             0.429
-1.1               NaN
wRC+               176
BsR                2.8
Fld               -2.3
-1.2               NaN
Off               62.3
Def                0.1
WAR                8.4
-1.3               NaN
ADP                1.1
playerid         10155
Points             521
Pos                 OF
Name: 0, dtype: object